In [1]:
import pandas as pd
import torch
import numpy as np
import random
from random import shuffle
from matplotlib import pyplot as plt
import seaborn as sns
from torch.utils.data import TensorDataset, DataLoader
import wget
import os
import aiohttp
import asyncio
np.random.seed(17)
random.seed(17)
torch.manual_seed(17)
from fpl import FPL
from player import Player
from team import Team
from data_processor import get_fpl, get_players, get_teams, get_training_datasets
import torch.nn as nn
import torch.optim as optim
seed = 5
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

In [2]:
fpl = await get_fpl()
opponent_feature_names = ["npxG","npxGA"]
player_feature_names = ["total_points", "ict_index", "clean_sheets", "saves", "assists", "goals_scored"]
teams = get_teams(team_feature_names=opponent_feature_names, visualize=False)
players = await get_players(player_feature_names, opponent_feature_names, visualize=False, num_players=640)
len(players)

639

In [3]:
if torch.cuda.device_count() >= 1:
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
else:
    torch.set_default_tensor_type(torch.DoubleTensor)

In [55]:
def normalize(x, is_scalar=False):
    '''
        Args 
            x - input 
        Returns
            normalized input
    '''
    if is_scalar:
        x = x.reshape((-1, )).double()
        means = torch.mean(x)
        stds = torch.std(x)
        return (x - means) / stds
    else:
        x = x.double()
        # (N, D, L)
        x = x.permute(0, 2, 1)
        input_means = torch.mean(x, dim=(0, 1))
        input_stds = torch.std(x, dim=(0, 1))
        x = (x - input_means) / (input_stds)
        x = x.permute(0, 2, 1)
        return x

def get_masked_training_datasets(players, teams, window_size=7, batch_size=50):
    '''
        Args
            players - list of players
            teams - list of teams
        Returns
            Train and test data loaders
            Input - (Batch_size, num_features, window)

            Build up datasets in numpy
            Make test train splits and feed it to pytorch
    '''
    X = []
    X_opponents = []
    for player in players:
        opponents = []
        for i in range(player.player_features.shape[1] - window_size):
            x = player.player_features[:,i:i+window_size]
            X.append(x)
            opponents.append((i+window_size-1, player.opponents[i+window_size-1]))
        for i, opponent in opponents:
            for team in teams:
                if team.name == opponent:
                    x_opponent = team.team_features[:,i-window_size+1:i+1]
                    if x_opponent.shape[1] != window_size:
                        x_opponent = np.zeros((x_opponent.shape[0], window_size))
                    X_opponents.append(x_opponent)

    X = np.array(X).astype(float)
    X_opponents = np.array(X_opponents).astype(float)
    X = np.concatenate((X, X_opponents), axis = 1)
    indices = np.random.permutation(range(len(X)))
    train_length = int(0.8 * len(X))
    X = torch.tensor(X).double()
    X = normalize(X)
    X_train, X_test = X[indices[:train_length]], X[indices[train_length:]] 
    train_loader = DataLoader(TensorDataset(X_train,), batch_size=batch_size)
    test_loader = DataLoader(TensorDataset(X_test,), batch_size=batch_size)
    return train_loader, test_loader


train_loader, test_loader = get_masked_training_datasets(players, teams)

In [56]:
normalize(torch.tensor([0, 1, 2, 3]), is_scalar=True)

tensor([-1.1619, -0.3873,  0.3873,  1.1619])

In [57]:
class AvgModel(nn.Module):
    def forward(self, x):
        '''
            Args 
                x - (N, D, L)
        '''
        return x[:,0,:].mean(dim=1).reshape((-1, ))


class PrevModel(nn.Module):
    def forward(self, x):
        '''
        '''
        return x[:,0,-1].reshape((-1, ))

class LinearModel(nn.Module):
    def __init__(self, window_size=4, num_features=5):
        super(LinearModel, self).__init__()
        self.dim = window_size * num_features
        self.fc1 = nn.Linear(self.dim, 1)
    
    def forward(self, x):
        x = x.reshape((x.shape[0], self.dim))
        return self.fc1(x).reshape((-1, ))

class RNNModel(nn.Module):
    def __init__(self, window_size=4, num_features=5, hidden_dim=128):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(num_features, hidden_dim, 5).double()
        self.fc = nn.Linear(hidden_dim, 1).double()

    def forward(self, x):
        '''
            Args
                x - shape (N, D, L)
        '''
        x = x.permute(2, 0, 1)
        h = self.rnn(x)
        o = self.fc(h[-1][-1])
        return o.reshape((-1, ))


input_tensor = torch.tensor([[0, 2, 4, 6], [1, 3, 5, 7]]).reshape((2, 1, 4)).double()
prev_model = PrevModel().double()
avg_model = AvgModel().double()
linear_model = LinearModel(num_features=1).double()
rnn_model = RNNModel(num_features=1)
print(avg_model.forward(input_tensor))
print(prev_model.forward(input_tensor))
print(linear_model.forward(input_tensor))
print(rnn_model.forward(input_tensor).shape)
print(len(train_loader))
sum(p.numel() for p in rnn_model.parameters())

tensor([3., 4.])
tensor([6., 7.])
tensor([-3.2932, -3.9300], grad_fn=<ViewBackward>)
torch.Size([2])
449


148993

In [27]:
next(linear_model.parameters()).is_cuda

True

In [60]:
def fit(model, train_loader, fixed_window=False, input_window=4, epochs=100, use_opponent_feature=False, len_opponent_features=2):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    model.train()
    for epoch in range(epochs):
        for (x,) in train_loader:
            optimizer.zero_grad()
            # inputs shape (Batch, D, window_size)
            if not fixed_window:
                input_window = np.random.choice([3+i for i in range(input_window-2)])
            inputs = x[:,:,:input_window]
            if not use_opponent_feature:
                inputs = inputs[:,:-len_opponent_features, :]
            outputs = x[:,0,input_window]
            predictions = model.forward(inputs)
            residual = (predictions - outputs)
            loss = (residual * residual).sum()
            loss.backward()
            optimizer.step()
    
def eval(model, test_loader, input_window=4, use_opponent_feature=False, len_opponent_features=2):
    sum_loss, count_loss = 0, 0
    sum_corr, count_corr = 0, 0
    model.eval()
    for (x,) in train_loader:
        inputs = x[:,:,:input_window]
        outputs = x[:,0,input_window]
        if not use_opponent_feature:
            inputs = inputs[:,:-len_opponent_features, :]
        predictions = (model.forward(inputs))
        assert(predictions.shape == outputs.shape)
        residual = (predictions - outputs)
        loss = (residual * residual).mean().item()
        outputs_numpy = outputs.detach().cpu().numpy()
        predictions_numpy = predictions.detach().cpu().numpy()
        corr = np.corrcoef(predictions_numpy, outputs_numpy)[0, 1]
        sum_loss += loss 
        count_loss += 1
        sum_corr += corr 
        count_corr += 1

    return sum_loss / count_loss, sum_corr / count_corr

- Are data driven methods better than common sense methods ?
    - Yes
- Is RNN really better than a linear model ?
    - Yes
- What is the optimal window size for prediction ?
    - input window 4
- Is output augmention helping ?
    - No

In [61]:


avg_model = AvgModel().double()
prev_model = PrevModel().double()

print(next(rnn_model.parameters()).is_cuda)
print('avg score', eval(avg_model, test_loader))
print('prev score', eval(prev_model, test_loader))
for input_window in [3, 4, 5, 6]:
    linear_model = LinearModel(window_size=input_window, num_features=6).double()
    rnn_model = RNNModel(num_features=6)
    rnn_model2 = RNNModel(num_features=6)
    fit(linear_model, train_loader, fixed_window=True, input_window=input_window)
    fit(rnn_model, train_loader, fixed_window=False, input_window=input_window)
    fit(rnn_model2, train_loader, fixed_window=True, input_window=input_window)
    print(input_window, 'linear', eval(linear_model, test_loader, input_window))
    print(input_window, 'rnn augmentation', eval(rnn_model, test_loader, input_window,))
    print(input_window, 'rnn no augmentation', eval(rnn_model2, test_loader, input_window))

True
avg score (0.8628282589288637, 0.4619136686957987)
prev score (1.2826155127124281, 0.3707182082707989)
3 linear (0.8072385402030575, 0.45936528665709175)
3 rnn augmentation (0.7812340813786567, 0.5088697098023881)
3 rnn no augmentation (0.7827364561006765, 0.508261400251003)
4 linear (0.7886968542288281, 0.47196552607701164)
4 rnn augmentation (0.7702099904458982, 0.5099374499841468)
4 rnn no augmentation (0.7674391947425359, 0.5049248345132665)
5 linear (0.805562546138286, 0.45467874504089295)
5 rnn augmentation (0.769536179693399, 0.5127957816429101)
5 rnn no augmentation (0.7535187849930188, 0.5101117220085389)
6 linear (0.7763684948605524, 0.47419534595012275)
6 rnn augmentation (0.7563990132737962, 0.5114440854564749)
6 rnn no augmentation (0.7355676707555406, 0.5148619359732576)


In [ ]:
- Is opponent feature helping

In [63]:


avg_model = AvgModel().double()
prev_model = PrevModel().double()

print(next(rnn_model.parameters()).is_cuda)
print('avg score', eval(avg_model, test_loader))
print('prev score', eval(prev_model, test_loader))
for input_window in [5]:
    linear_model = LinearModel(window_size=input_window, num_features=8).double()
    rnn_model = RNNModel(num_features=8)
    rnn_model2 = RNNModel(num_features=8)
    fit(linear_model, train_loader, fixed_window=True, input_window=input_window, use_opponent_feature=True)
    fit(rnn_model, train_loader, fixed_window=False, input_window=input_window, use_opponent_feature=True)
    fit(rnn_model2, train_loader, fixed_window=True, input_window=input_window, use_opponent_feature=True)
    print(input_window, 'linear', eval(linear_model, test_loader, input_window, use_opponent_feature=True))
    print(input_window, 'rnn augmentation', eval(rnn_model, test_loader, input_window, use_opponent_feature=True))
    print(input_window, 'rnn no augmentation', eval(rnn_model2, test_loader, input_window, use_opponent_feature=True))

True
avg score (0.8628282589288637, 0.4619136686957987)
prev score (1.2826155127124281, 0.3707182082707989)
5 linear (0.7726572124156315, 0.4872543556534628)
5 rnn augmentation (0.7687572520902652, 0.5087720206309768)
5 rnn no augmentation (0.7554680780470515, 0.5094116710954579)
